In [1]:
import requests
import os
import sys
import pandas as pd
import json

/tmp/ipykernel_42884/2719384636.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from ff14_utils import *

In [4]:
if 'google.colab' in sys.modules:
    root_dir = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'ff14')
    data_dir = os.path.join(root_dir, 'data')
else:
    root_dir = os.getcwd()
    data_dir = os.path.join(root_dir, 'data')

In [5]:
ITEM_CATEGORY_CONFIG_MAP = json.load(open(os.path.join(data_dir, 'item_category_config_map.json')))

In [6]:
data_centers = get_data_centers()
worlds = get_worlds()

target_worlds = []

for world in worlds:
    if world['id'] in data_centers[5]['worlds']:
        target_worlds.append((world['id'], world['name'], {}))
        #print(world['id'], world['name'])

for world in target_worlds:
    tax_rates = get_tax_rates(world[0])
    target_worlds[target_worlds.index(world)] = (world[0], world[1], tax_rates)

In [7]:
path = os.path.join(data_dir, 'Items_v3.csv')

df = pd.read_csv(path, header=0)

In [8]:
categories = ITEM_CATEGORY_CONFIG_MAP.keys()
print(list(categories))

['All', 'Pugilist Weapons', 'One-handed Swords', 'Greataxes', 'Bows', 'Polearms', 'One-handed Conjurer Arms', 'Two-handed Conjurer Arms', 'One-handed Thaumaturge Arms', 'Two-handed Thaumaturge Arms', 'Grimoires', 'Shields', "Carpenter's Primary Tool", "Carpenter's Secondary Tool", "Blacksmith's Primary Tool", "Blacksmith's Secondary Tool", "Armorer's Primary Tool", "Armorer's Secondary Tool", "Goldsmith's Primary Tool", "Goldsmith's Secondary Tool", "Leatherworker's Primary Tool", "Leatherworker's Secondary Tool", "Weaver's Primary Tool", "Weaver's Secondary Tool", "Alchemist's Primary Tool", "Alchemist's Secondary Tool", "Culinarian's Primary Tool", "Culinarian's Secondary Tool", "Miner's Primary Tool", "Miner's Secondary Tool", "Botanist's Primary Tool", "Botanist's Secondary Tool", "Fisher's Primary Tool", 'Fishing Tackle', 'Head Armor', 'Body Armor', 'Leg Armor', 'Hand Armor', 'Foot Armor', 'Waist Armor', 'Necklaces', 'Earrings', 'Bracelets', 'Rings', 'Potions', 'Ingredients', 'Foo

In [9]:
fishs = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Fish']]['#'].to_list()
potions = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Potions']]['#'].to_list()
foods = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Food']]['#'].to_list()
ingredients = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Ingredients']]['#'].to_list()
shards = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Crystals']]['#'].to_list()
furnishings = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Furnishings']]['#'].to_list()
materias = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Materia']]['#'].to_list()

miscellaneous_categories = ['Metals', 'Lumber', 'Cloth', 'Leather', 'Bone', 'Alchemical Materials', 'Dyes', 'Parts']
miscellaneous = df[df['ItemUICategory'].isin([ITEM_CATEGORY_CONFIG_MAP[category] for category in miscellaneous_categories])]['#'].to_list()

In [10]:
target_items = foods

### History keys :
* itemID
* worldID
* lastUploadTime (in milliseconds since the UNIX epoch)
* entries (dict)
* stackSizeHistogram (nombre de ventes effectuer avec une quantité donnée)
* stackSizeHistogramNQ (idem pour les NQ)
* stackSizeHistogramHQ (idem pour les HQ)
* regularSaleVelocity (The average number of sales per day)
* nqSaleVelocity (idem pour les NQ)
* hqSaleVelocity (idem pour les HQ)
* worldName

### Entries keys :
* hq (bool)
* pricePerUnit (Gil)
* quantity
* buyerName
* onMannequin (bool)
* timestamp (in seconds since the UNIX epoch)

In [11]:
seconds_in_a_day = 60 * 60 * 24
milliseconds_in_a_day = seconds_in_a_day * 1000

In [12]:
history = get_history(83, 4749, entriesToReturn=2000, statsWithin=14 * milliseconds_in_a_day, entriesWithin=14 * seconds_in_a_day)

In [28]:
df_entries = pd.DataFrame(columns=['hq', 'price', 'quantity', 'timestamp'])

entries = history['entries']

for entry in entries:
    df_temp = pd.DataFrame({'hq': [entry['hq']], 'price': [entry['pricePerUnit']], 'quantity': [entry['quantity']], 'timestamp': [entry['timestamp']]})
    df_entries = pd.concat([df_entries, df_temp])

df_entries['timestamp'] = pd.to_datetime(df_entries['timestamp'], unit='s')
df_entries = df_entries.sort_values(by='timestamp')
df_entries = df_entries.reset_index(drop=True)

In [29]:
df_entries.head(10)

,hq,price,quantity,timestamp
0,True,900,10,2024-02-08 18:36:39
1,True,950,10,2024-02-08 18:36:42
2,True,950,5,2024-02-08 18:36:44
3,True,2000,11,2024-02-08 18:36:50
4,True,950,5,2024-02-08 18:36:54
5,True,912,1,2024-02-08 18:36:57
6,True,899,2,2024-02-08 18:36:58
7,False,150,26,2024-02-08 18:48:13
8,False,320,81,2024-02-09 22:46:50
9,True,1500,1,2024-02-10 14:32:53


In [35]:
import plotly.express as px

# plot the price history of hq and nq items
# show the timestamp on the x-axis as a day of the week

fig = px.line(df_entries, x='timestamp', y='price', color='hq', title='Price History')
fig.update_xaxes(tickformat="%A")
fig.show()

In [ ]:
df_history = pd.DataFrame(columns=['World', 'Item', 'Price', 'Quantity', 'Timestamp'])

for world in target_worlds:
    # separate the list of items into chunks of 100
    chunks = [target_items[x : x + 100] for x in range(0, len(target_items), 100)]
    for chunk in chunks:
        data = get_items(world, chunk)
        if data:
            for item in data["items"]:
                item_data = [data['items'][item][key] for key in item_keys]
                df_items = pd.concat([df_items, pd.DataFrame([item_data], columns=item_keys)])
        else:
            print("No data for", world)